<a href="https://colab.research.google.com/github/LiShun522/tibame_1224/blob/main/250113_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas
```python
1. 從無 生有一個表格
  Step1. 你要準備一個空的List
  Step2. 把你的每一筆資料準備成一個Dict, 放進List裡
  Step3. 轉成DataFrame儲存

  [damu, damu...]


```

In [49]:
import urllib.request as req
import bs4 as bs
# Step 0.
import pandas as pd

# Step1. 準備一個空的List
table = []
page = 1
while True:
    print("頁數:", page)
    url = "https://tabelog.com/tw/tokyo/rstLst/{}/?SrtT=rt".format(page)
    page = page + 1
    response = req.urlopen(url)
    html = bs.BeautifulSoup(response)

    r_list = html.find_all("div", {"class":"list-rst__body"})
    if len(r_list) == 0:
        break

    for r in r_list:
        en_name = r.find("a", {"class":"list-rst__rst-name-target"})
        en_name_href = en_name["href"]
        en_name_text = en_name.text
        area = r.find("div", {"class":"list-rst__area-genre"})
        area_text = area.text.strip()
        intro = r.find("p", {"class":"list-rst__pr-title"})
        if intro == None:
            intro_text = "無"
        else:
            intro_text = intro.text.strip()
        rating = r.find("span", {"class":"c-rating__val"})
        rating_text = rating.text
        prices = r.find_all("span", {"class":"c-rating-v3__val"})
        dinner_price = prices[0].text
        lunch_price = prices[1].text
        holiday = r.find("span", {"class":"list-rst__holiday-text"})
        holiday_text = holiday.text

        print(rating_text)
        print(en_name_text)
        print(en_name_href)
        print(area_text)
        print(intro_text)
        print(holiday_text)
        print("晚間價錢:", dinner_price, "午間價錢:", lunch_price)
        print("-" * 30)

        # Step2. 每一筆資料就是一個字典
        data = {
            "name(en)": en_name_text,
            "url": en_name_href,
            "area": area_text,
            "intro": intro_text,
            "rating": rating_text,
            "holiday": holiday_text
        }
        table.append(data)

# Step3. 轉成DataFrame儲存
df = pd.DataFrame(table,
          columns=["name(en)", "url", "area", "intro", "rating","holiday"])
df.to_csv("tabelog.csv", encoding="utf-8")


串流輸出內容已截斷至最後 5000 行。
3.78
SIROTAE
https://tabelog.com/tw/tokyo/A1308/A130801/13002850/
赤坂見附車站 / 蛋糕, 咖啡店, 喫茶店
無
星期一、星期日
晚間價錢: - 午間價錢: JPY 1,000～JPY 1,999
------------------------------
頁數: 30
3.78
Naniwaya Sohonten
https://tabelog.com/tw/tokyo/A1307/A130702/13002775/
麻布十番車站 / 日式甜點店, 鯛魚燒・大判燒, 炒麵
無
星期二、星期三
晚間價錢: - 午間價錢: ～JPY 999
------------------------------
3.78
Niboshi Tsukemen Miyamoto
https://tabelog.com/tw/tokyo/A1315/A131503/13180862/
蒲田車站 / 沾麵, 拉麵
[從蒲田車站步行4分鐘]獨一無二的拉麵！ ！
星期日
晚間價錢: JPY 1,000～JPY 1,999 午間價錢: JPY 1,000～JPY 1,999
------------------------------
3.78
Tsukesoba Kanda Katsumoto
https://tabelog.com/tw/tokyo/A1310/A131003/13192433/
神保町車站 / 沾麵, 拉麵
無
星期日
晚間價錢: - 午間價錢: ～JPY 999
------------------------------
3.78
Mentokoro Ginzasa
https://tabelog.com/tw/tokyo/A1301/A130103/13119563/
築地市場車站 / 拉麵, 沾麵
無
星期日、節假日
晚間價錢: JPY 1,000～JPY 1,999 午間價錢: JPY 1,000～JPY 1,999
------------------------------
3.78
Ramen Kai
https://tabelog.com/tw/tokyo/A1311/A131103/13191776/
藏前車站 / 拉麵, 沾麵
無
-
晚間價

# Pandas 基本操作

```python
直 得叫行(column) 列(row)

1. 取 column
2. 取 row
3. filter
4. apply
5. 無中生有一個表格(下面)
```

In [10]:
practice = pd.DataFrame(
    [
      [1, 2, 3],
      [4, 5, 6],
      [7, 8, 9]
    ],
    columns = ["a", "b", "c"],
    index= ["c", "d", "e"])
practice

,a,b,c
c,1,2,3
d,4,5,6
e,7,8,9


In [16]:
# 取行(column): 字典
print(practice["a"])    # 取 單列
practice[["a", "c", "b"]] # 可以自己選排列

c    1
d    4
e    7
Name: a, dtype: int64


,a,c,b
c,1,3,2
d,4,6,5
e,7,9,8


In [14]:
# pandas 主要型態分成兩種
# 1. DataFrame: 表格型態，有行有列(2D)
# 2. Series: List型態，單行單列(1D)
print(type(practice["a"]))
print(type(practice[["a", "c"]]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [20]:
# 取列(row)
# 1. 看他實際第幾列(iloc): list, 把0..2當key  !! 絕對值，不會被改
# 2. 看列標籤(loc): 字典, 列標籤當key
# 專屬值
practice.loc["d"]
practice.iloc[1]

# 平常使用loc會剛好對那是因為可能index列標籤沒設定，為預設

,d
a,4
b,5
c,6


In [21]:
# 過濾(filter)原理: 把跟妳列的筆數一樣多的True False帶給她[]
# 對到True的就會留下, 對到False的就會丟掉
fil = [True, False, True]
practice[fil]

,a,b,c
c,1,2,3
e,7,8,9


In [45]:
df["rating"] = df["rating"].astype("float")
# fil = df["rating"] >= 4.0
# df[fil]

# 妳可以做條件組合 and->& or->|
fil = (df["rating"] <= 4.1) & (df["rating"] >= 3.8)
df[fil]

,name(en),url,area,intro,rating
148,HATSUNEZUSHI,https://tabelog.com/tw/tokyo/A1315/A131503/130...,蒲田車站 / 壽司,充滿渾身解數和氣魄的超乎想象的握壽司。,4.1
149,Tori Chataro,https://tabelog.com/tw/tokyo/A1303/A130301/131...,澀谷車站 / 烤雞肉串,無,4.1
150,Akasaka Kitafuku,https://tabelog.com/tw/tokyo/A1308/A130801/131...,赤坂車站 / 螃蟹,是煮著吃，還是烤著吃?在只有3間單間的奢華空間裡，盡情品嘗一杯螃蟹。,4.1
151,unis,https://tabelog.com/tw/tokyo/A1308/A130802/132...,"虎之門Hills車站 / 法式料理, 創新","使用來自全國各地的時令食材制作的Chef'sTable,讓您的Hare日變得華麗",4.1
152,YORONIKU TOKYO AZABUDAIHILLS,https://tabelog.com/tw/tokyo/A1307/A130701/132...,六本木一丁目車站 / 燒肉,無,4.1
...,...,...,...,...,...
525,Think,https://tabelog.com/tw/tokyo/A1311/A131105/132...,"日暮里車站 / 麵包, 西式甜點",無,3.8
526,Four Seasons,https://tabelog.com/tw/tokyo/A1301/A130101/130...,銀座車站 / 酒吧,無,3.8
527,Sushi Saitou Azabudai,https://tabelog.com/tw/tokyo/A1307/A130701/132...,神谷町車站 / 壽司,無,3.8
528,Sushi Imamura,https://tabelog.com/tw/tokyo/A1316/A131602/132...,白金台車站 / 壽司,無,3.8


In [51]:
# # 如果要接後續或者妳要查詢妳的資料, 妳就要思考一下妳的資料處理
# table = []
# for h in df["holiday"]:
#     data = {
#         "星期一":True,
#         "星期二":True,
#         "星期三":True,
#         "星期四":True,
#         "星期五":True,
#         "星期六":True,
#         "星期日":True
#     }
#     check = data.keys()
#     for c in check:
#         if c in h:
#             data[c] = False
#     table.append(data)
# df2 = pd.DataFrame(table)
# total = pd.concat([df, df2], axis=1)
# # 過濾: 禮拜日有開, 要大於3.5
# fil = (total["星期日"] == True) & (total["rating"].astype("float") >= 3.7)
# total[fil]

,name(en),url,area,intro,rating,holiday,星期一,星期二,星期三,星期四,星期五,星期六,星期日
0,Nihonbashi Kakigaracho Sugita,https://tabelog.com/tw/tokyo/A1302/A130204/130...,水天宮前車站 / 壽司,將江戶前的精髓傳承至今的壽司。持續進化的職人技藝和細衹周到，極具魅力的正統派江戶前壽司。,4.67,-,True,True,True,True,True,True,True
3,Aka,https://tabelog.com/tw/tokyo/A1302/A130202/132...,三越前車站 / 西班牙料理,精緻的西班牙菜，閃耀著日本的美感和東主廚的獨創性。,4.61,-,True,True,True,True,True,True,True
4,Sushi Arai,https://tabelog.com/tw/tokyo/A1301/A130101/131...,銀座車站 / 壽司,在擔任久兵衛和壽司師傅后，他在 33 歲時獨立。 堅持「握壽司」的新一代銀座壽司,4.59,星期三,True,True,False,True,True,True,True
7,Ao,https://tabelog.com/tw/tokyo/A1307/A130701/132...,"六本木車站 / 創新, 法式料理",無,4.57,-,True,True,True,True,True,True,True
8,PELLEGRINO,https://tabelog.com/tw/tokyo/A1303/A130302/130...,廣尾車站 / 義大利料理,想最大限度地傳達食材的味道、香味、時令,4.57,-,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,Hasegawa,https://tabelog.com/tw/tokyo/A1312/A131201/130...,兩國車站 / 炸豬排,無,3.72,-,True,True,True,True,True,True,True
1194,Zopf Curry Pan Senmon Ten Gransta Ten,https://tabelog.com/tw/tokyo/A1302/A130201/132...,"東京車站 / 麵包, 咖喱",無,3.72,-,True,True,True,True,True,True,True
1195,Ramen Jiro Meguro Ten,https://tabelog.com/tw/tokyo/A1316/A131601/130...,目黑車站 / 拉麵,無,3.72,-,True,True,True,True,True,True,True
1197,Dame na Rinjin Ningyocho Ten,https://tabelog.com/tw/tokyo/A1302/A130204/132...,"人形町車站 / 拉麵, 油麵/拌麵, 沾麵",人形町站旁邊！ ◆享受嚴選食材製作的精緻拉麵和土鍋飯♪外帶也可以◎,3.72,-,True,True,True,True,True,True,True


# 函式 是一個形態 !!!!

In [63]:
# Series.apply(轉換方式)
# 原有值 => 轉換方式 => 新值
# 轉換方式就是一個函式， 不加小括號
df["rating"] = df["rating"].astype("float")
df["rating"].apply(round) # round不加小括號， pd會幫你加，可參照下一段 # 功能是四捨五入

,rating
0,5
1,5
2,5
3,5
4,5
...,...
1195,4
1196,4
1197,4
1198,4


In [62]:
# 函式是形態(????)
# 以前: 我跟你說每個功能都要加小括號， 就算沒帶入參數也要加
# Everything is an object: 每個東西都是有形態的，每個形態都有這個形態可以做的操作
# list 操作:[0] [-1]  dict 操作: [key]
# round(4.2)??
# round是一個形態 (4.2)是這個形態的操作
# round是一個函式的步驟集合形態  (4.2)才是真的把這步驟執行完成得到結果
#      刷牙步驟               開始刷
# 函是形態: 還沒開始被打開的錦囊
# 加小括號: 把錦囊打開， 執行並得到結果
b = round
b(5.5)
import math
def func(state):
  if state == 0:
    return math.gcd # func(0)時候使用，最大公因數
  else:
    return math.lcm # func(1)時候使用，最小公倍數

func(0)(6, 9), func(1)(6, 9)

(3, 18)

In [72]:
import pandas as pd
df = pd.read_csv("tabelog.csv", encoding = "utf-8")
df["area"]
def func(s):
  real_area = s.split(" / ")[0] # 從 "空白全形/空白" 開始切
  return real_area.strip()     # 將左右兩邊空白去除
# func("水天宮前車站 / 壽司")
df["area"] = df["area"].apply(func)        # 還不知道帶甚麼進去，所以還不設置小括號
df["area"]

,area
0,水天宮前車站
1,御成門車站
2,赤羽橋車站
3,三越前車站
4,銀座車站
...,...
1195,目黑車站
1196,溜池山王車站
1197,人形町車站
1198,藏前車站


In [71]:
df["area"].value_counts()

,count
area,
銀座車站,50
六本木車站,37
廣尾車站,35
麻布十番車站,27
惠比壽車站,25
...,...
千石車站,1
板橋區政府前車站,1
石神井公園車站,1


In [75]:
def func(s):
  if "惠比壽車站" in s:
    return True
  else:
    return False
fil = (df["area"].apply(func)) & (df["rating"].astype("float") >= 3.9)
df[fil]

,Unnamed: 0,name(en),url,area,intro,rating,holiday
39,39,Gastronomy Joel Robuchon,https://tabelog.com/tw/tokyo/A1303/A130302/130...,惠比壽車站,品味Joel Robuchon先生精髓的Gran Maison。,4.40,-
51,51,Ebisu Yoroniku,https://tabelog.com/tw/tokyo/A1303/A130302/132...,惠比壽車站,美食之街，佇立在惠比壽的肉拉面魅力不減~新一代的肉割烹~,4.35,-
58,58,WASA,https://tabelog.com/tw/tokyo/A1303/A130302/132...,惠比壽車站,最大限度地發揮食材的味道，獨一無二的現代中式。,4.34,-
136,136,Eiki,https://tabelog.com/tw/tokyo/A1303/A130302/132...,惠比壽車站,東京新時代烤鸡肉串的代表，擁有名店的血統。,4.13,星期日
271,271,Ebisu Endo,https://tabelog.com/tw/tokyo/A1303/A130302/132...,惠比壽車站,無,3.93,-
288,288,LESS Labo,https://tabelog.com/tw/tokyo/A1303/A130302/132...,惠比壽車站,無,3.91,星期三
308,308,LESS by Gabriele Riva & Kanako Sakakura,https://tabelog.com/tw/tokyo/A1303/A130302/132...,惠比壽車站,無,3.90,星期三


In [79]:
# Series.apply(func, convert_dtype=<no_default>, args=(), *, by_row='compat', **kwargs)
def func(s, name):
  if name == s: # 本來是 in 改為 == 因為有可能有不同類似參數
    return True
  else:
    return False
fil = df["area"].apply(func, args=["銀座車站"])
df[fil]

,Unnamed: 0,name(en),url,area,intro,rating,holiday
4,4,Sushi Arai,https://tabelog.com/tw/tokyo/A1301/A130101/131...,銀座車站,在擔任久兵衛和壽司師傅后，他在 33 歲時獨立。 堅持「握壽司」的新一代銀座壽司,4.59,星期三
11,11,Sawada,https://tabelog.com/tw/tokyo/A1301/A130101/130...,銀座車站,無,4.55,星期一、星期四、星期日
25,25,L'OSIER,https://tabelog.com/tw/tokyo/A1301/A130101/130...,銀座車站,傳統與革新相協調，日本在世界上引以為傲的最高峰Grand Maison,4.46,星期一、星期日
38,38,Nikuya Tanaka,https://tabelog.com/tw/tokyo/A1301/A130101/132...,銀座車站,一切都是為了顧客的笑容,4.41,星期日
55,55,ESqUISSE,https://tabelog.com/tw/tokyo/A1301/A130101/131...,銀座車站,未來最新的法式料理,4.34,星期三
68,68,Sushi Suzuki,https://tabelog.com/tw/tokyo/A1301/A130101/131...,銀座車站,在凜然的日式空間裡享受極品的一貫。享受精衹細膩的正統派江戶前手握壽司,4.31,星期一
96,96,ZURRIOLA,https://tabelog.com/tw/tokyo/A1301/A130101/131...,銀座車站,在尋找與日本料理的相似之處後，我們發現了展現季節表達的「現代西班牙菜」。,4.25,星期一
105,105,USHIGORO S. GINZA,https://tabelog.com/tw/tokyo/A1301/A130101/132...,銀座車站,今年也非常感謝您的支持！,4.24,-
120,120,subin,https://tabelog.com/tw/tokyo/A1301/A130101/132...,銀座車站,享受「原汁原味」...對肉的追求，邂逅但馬金槍魚－終極涮涮鍋,4.20,星期六、星期日
131,131,L'ecrin,https://tabelog.com/tw/tokyo/A1301/A130101/130...,銀座車站,留在記憶中的追求食材的“美麗”的一盤，與上等的空間和服務一起,4.14,-


# 抓出關鍵詞
```python
1. 西方型: 英文 字母=>詞彙=>片語(pharse)
2. 東方型: 中文     字=>詞彙

分詞: 我喜歡吃甜食 => 我/喜歡/吃/甜食

哪個詞彙在整段話裡最重要:
1. 出現次數(TF): 高, 重要性越高
2. 常用程度(IDF): 低, 重要性越高

詞彙重要程度:
# TF-IDF = 出現次數 * log(1/常用程度) 不寫log是因為感受度前面有後面要非常大
TF-IDF = 出現次數 * (1/常用程度)

!!! jibba函式庫: 預先統計了常用程度
DF = 詞彙出現在幾篇裡面/收集了夠多篇文章
!!!        : 分詞
利用詞彙庫做出初步劃分->(沒在詞彙庫/混淆)上下文推測哪種分法最合理

## Jieba 不需要一個很龐大的東西去解決(GPT之類)
### 拉出關鍵詞的函式庫
```


In [80]:

df["intro"]

,intro
0,將江戶前的精髓傳承至今的壽司。持續進化的職人技藝和細衹周到，極具魅力的正統派江戶前壽司。
1,將食材的美味發揮到極致的日本料理大師。
2,繼承了名店的靈魂，將極品魚的精湛技藝發揮到極衹的現代江戶前壽司的未來
3,精緻的西班牙菜，閃耀著日本的美感和東主廚的獨創性。
4,在擔任久兵衛和壽司師傅后，他在 33 歲時獨立。 堅持「握壽司」的新一代銀座壽司
...,...
1195,無
1196,無
1197,人形町站旁邊！ ◆享受嚴選食材製作的精緻拉麵和土鍋飯♪外帶也可以◎
1198,無


In [82]:
news = """
饗饗信義店爆發食物中毒案，累計59人就醫通報，目前該店已恢復營業，針對感染可能，北市衛生局食藥科長林冠蓁表示，個案中有超過7成都有食用過生魚片，且依照潛伏期和相關症狀看來，與諾羅病毒比較相似，但仍需要等待後續報告出爐才能確認。

饗饗食物中毒案累計通報59名個案，昨日核准業者復業，林冠蓁指出，就醫民眾共食的食材包含生魚片、生蠔及干貝，目前已採樣環境檢體4件、人體檢體4件，結果最快1到2周後出爐，而59名通報案中，分別有42人吃生魚片、21人食用干貝、5人食用生蠔，有超過7成有吃過生魚片。



林冠蓁說，根據復業報告中推測，可能是生鮮食材遭受諾羅病毒、病原性大腸桿菌、沙門氏菌等污染，加上目前通報個案的症狀和潛伏期都和諾羅病毒類似，高機率是諾羅病毒感染，但詳細原因仍需待調查報告出爐。

如符合食品中毒病因物質及原因食品，將依違反《食品安全衛生管理法》第15條及第44條處6萬到2億元罰鍰。
"""

In [83]:
import jieba.analyse
# jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
jieba.analyse.extract_tags(news, topK=20, withWeight=False, allowPOS=())

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.882 seconds.
DEBUG:jieba:Loading model cost 0.882 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


['諾羅',
 '59',
 '魚片',
 '林冠',
 '個案',
 '超過',
 '過生',
 '潛伏期',
 '症狀',
 '報告',
 '出爐',
 '通報',
 '食材',
 '干貝',
 '檢體',
 '病毒',
 '食用',
 '食物中毒',
 '北市',
 '信義店']

In [84]:
import jieba.analyse
import urllib.request as req

url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big" # 導入繁體中文

req.urlretrieve(url, "dict")
jieba.set_dictionary("dict")


In [86]:
import jieba.analyse
jieba.analyse.extract_tags(news)

['通報',
 '生魚片',
 '諾羅',
 '59',
 '個案',
 '出爐',
 '林冠',
 '累計',
 '就醫',
 '超過',
 '潛伏期',
 '症狀',
 '報告',
 '復業',
 '干貝',
 '病毒',
 '食用',
 '食物中毒',
 '食品',
 '信義']

In [87]:
# 設置
import jieba
import urllib.request as req
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
req.urlretrieve(url, "dict")
jieba.set_dictionary("dict")
# 使用者辭典
extra_url = "https://github.com/Elwing-Chou/tibame1224/raw/refs/heads/main/extra.dict.txt"
req.urlretrieve(extra_url, "extra.dict.txt")
jieba.load_userdict("extra.dict.txt")

Building prefix dict from /content/dict ...
DEBUG:jieba:Building prefix dict from /content/dict ...
Dumping model to file cache /tmp/jieba.u08cf431985a91c6bc29a3dba35bb8b2f.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.u08cf431985a91c6bc29a3dba35bb8b2f.cache
Loading model cost 1.574 seconds.
DEBUG:jieba:Loading model cost 1.574 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [88]:
" ".join(jieba.cut(news))

'\n 饗饗 信義 店 爆發 食物中毒 案 ， 累計 59 人 就醫 通報 ， 目前 該店 已 恢復 營業 ， 針對 感染 可能 ， 北 市 衛生局 食藥 科長 林冠 蓁 表示 ， 個案 中有 超過 7 成都 有 食用 過 生魚片 ， 且 依照 潛伏期 和 相關 症狀 看來 ， 與 諾羅病毒 比較 相似 ， 但 仍 需要 等待 後續 報告 出爐 才能 確認 。 \n \n 饗饗 食物中毒 案 累計 通報 59 名 個案 ， 昨日 核准 業者 復業 ， 林冠 蓁 指出 ， 就醫 民眾 共食 的 食材 包含 生魚片 、 生蠔 及 干貝 ， 目前 已 採樣 環境 檢體 4 件 、 人 體檢 體 4 件 ， 結果 最快 1 到 2 周 後 出爐 ， 而 59 名 通報 案中 ， 分別 有 42 人 吃 生魚片 、 21 人 食用 干貝 、 5 人 食用 生蠔 ， 有 超過 7 成有 吃過 生魚片 。 \n \n   \n \n 林冠 蓁 說 ， 根據 復業 報告 中 推測 ， 可能 是 生 鮮食 材 遭受 諾羅病毒 、 病原 性 大腸桿菌 、 沙門氏菌 等 污染 ， 加上 目前 通報 個案 的 症狀 和 潛伏期 都 和 諾羅病毒 類似 ， 高 機率 是 諾羅病毒 感染 ， 但 詳細 原因 仍 需待 調查報告 出爐 。 \n \n 如 符合 食品 中毒 病因 物質 及 原因 食品 ， 將依 違反 《 食品 安全衛生 管理法 》 第 15 條 及第 44 條處 6 萬到 2 億元 罰 鍰 。 \n'

In [89]:
import jieba.analyse
jieba.analyse.extract_tags(news, allowPOS=["n", "nr", "ns", "nt", "nz"])

['生魚片',
 '諾羅病毒',
 '林冠',
 '饗饗',
 '生蠔',
 '干貝',
 '食用',
 '食品',
 '食藥',
 '民眾',
 '共食',
 '食材',
 '檢體',
 '大腸桿菌',
 '沙門氏菌',
 '條處',
 '管理法',
 '原因',
 '及第',
 '病原']